In [1]:
import DWM1_Tokenizer
import DWM2_Global_Token_Replace_NewDict
import DWM3_Entropy_Cluster
# Settings for Tokenizer
inputSampleName = r'S8'
delimiter=','
hasHeader = True
tokenizerType = 'Splitter'
removeDuplicateTokens = True

# Setting for global replacement DWM2
runReplacement = True
minFreqStdToken = 5
minLenStdToken = 3
maxFreqErrToken = 3

# Cluster Settings


DWM1_Tokenizer.tokenizeCycle(inputSampleName, delimiter, hasHeader, removeDuplicateTokens, tokenizerType)
prefixString = inputSampleName +','+tokenizerType+','+str(removeDuplicateTokens)
fileToCluster = inputSampleName + '-Tokenized.txt'
if runReplacement:
    DWM2_Global_Token_Replace_NewDict.globalReplace(inputSampleName, minFreqStdToken, minLenStdToken, maxFreqErrToken)
    fileToCluster = inputSampleName + '-TokenReplace.txt'
    prefixString = prefixString+','+str(minFreqStdToken)+','+str(minLenStdToken)+','+str(maxFreqErrToken)
else:
    prefixString = prefixString+',0,0,0'

# Cluster fixed settings
epsilonIterate = 0.0
runClusterMetrics = False
runFinalMetrics = True
createFinalJoin = False

Tokenizing References
Total References= 1000
Total Tokens Found = 13025
Total Tokens Output = 12511
Dictionary Load = 67077
<class 'dict'>
Global Replace Cycle
Total References= 1000
Total Tokens= 12511
Unique Tokens= 3978
Minimum Frequency of Standard Token =  5
Minimum Length of Standard Token =  3
Maximum Frequency of Error Token =  3
References Processed= 1000
Sorted Token Size = 3978
Clean Token Size = 1776
*Stop Replacements here
References Processed =  1000
Total Replacement Pairs = 240
Tokens Read = 12511
Tokens Changed =  256
References Changed = 233


In [2]:
status="done"

In [3]:
print(status)

done


In [4]:
# # Cluster Robot Settings
# muStart = 0.60
# muEnd = 0.60
# muIncr = 0.05
# betaStart = 14
# betaEnd = 14
# betaIncr = 1
# sigmaStart = 145
# sigmaEnd = 145
# sigmaIncr = 1
# epsilonStart = 35
# epsilonEnd = 35
# epsilonIncr = 1
# mu = muStart
# while mu <= muEnd:
#     beta = betaStart
#     while beta <= betaEnd:
#         sigma = sigmaStart
#         while sigma <= sigmaEnd:
#             epsilon = epsilonStart
#             while epsilon <= epsilonEnd:
#                 suffixString =','+str(epsilon)+','+str(epsilonIterate)
#                 resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, beta, sigma, mu, muIncr,epsilon,  \
#                                                      epsilonIterate,runClusterMetrics, runFinalMetrics, createFinalJoin, hasHeader)
#                 seg0a = ','+str(resultsList[0])+','+str(resultsList[1])+','+str(resultsList[2])+','+str(resultsList[3])
#                 seg0b =','+str(resultsList[4])+','+str(resultsList[5])+','+str(resultsList[6])
#                 seg1 = ','+str(beta)+','+str(sigma)+','+str(mu)+ ',' +str(muIncr)
#                 seg2 = ','+str(resultsList[7])+','+str(resultsList[8])+','+str(resultsList[9])
#                 outString = prefixString+seg0a+seg0b+suffixString+seg1+seg2+'\n'
#                 resultsFile = open('RobotResults.txt','a+')
#                 resultsFile.write(outString)
#                 resultsFile.close()
#                 epsilon = epsilon + epsilonIncr
#             sigma = sigma + sigmaIncr
#         beta = beta + betaIncr
#     mu = mu + muIncr


 

## Bayesian Optimization of Hyperparameters

In [4]:
import numpy as np

Create a function of the parameters that returns the F score

In [5]:
def entropy_F1(beta=14,sigma=145,mu=0.6,muIncr=0.05,epsilon=35,epsilonIterate=0):
    resultsList = DWM3_Entropy_Cluster.driver(fileToCluster, 
                                  beta, 
                                  sigma, 
                                  mu, 
                                  muIncr,
                                  epsilon,
                                  epsilonIterate,
                                  runClusterMetrics, 
                                  runFinalMetrics, 
                                  createFinalJoin, 
                                  hasHeader)
    return resultsList[9]

In [7]:
entropy_F1(mu=0.6,muIncr=0.05,beta=14,sigma=145,epsilon=35,epsilonIterate=0.0)


>>>>>>>>>>>>
Starting Iteration mu= 0.6
Starting Iteration epsilon= 35
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 1715
NewMatrix Iterateblocks Total Time = 1.3762334799394011

Precision = TP / (TP + FP) ......................... = 0.78797468
Recall = TP / (TP + FN) ............................ = 0.88801712
F-Measure .......................................... = 0.83501006


0.83501006

Create functions for variable transformation.

In [8]:
#Softplus to transform real domain into positive
def softplus(x):
    return np.log(1+np.exp(x))

def inv_softplus(x):
    return np.log(np.exp(x)-1)

In [9]:
inv_softplus(1)

0.541324854612918

In [10]:
#Logit to transform real domain into 0-1
def logit(x):    
    return np.log(x+1e-54)-np.log(1-x+1e-54)

def inv_logit(x):
    res=np.exp(x)/(np.exp(x)+1)-2*1e-54
    if res<1e-54:
        res=0
    return res

Import BO library:

In [11]:
from bayes_opt import BayesianOptimization

Create dictionary bounds for the parameters:

In [12]:
import os, sys

In [13]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [14]:
with HiddenPrints():
    print("This will not be printed")

print("This will be printed as before")

This will be printed as before


In [15]:
#Take parameters in transformed space and return f score in transformed space
def entropy_F1_trx(beta_sp,sigma_sp,mu_logit,muIncr_logit,epsilon_sp,epsilonIterate_logit=logit(0)):
    beta_pos=int(softplus(beta_sp))
    sigma_pos=int(softplus(sigma_sp))
    mu_01=inv_logit(mu_logit)
    muIncr_01=inv_logit(muIncr_logit)
    epsilon_pos=softplus(epsilon_sp)
    epsilonIterate_01=inv_logit(epsilonIterate_logit)
    
    print("Mu:%.2f|Mu+:%.2f|Beta:%.2f|Sigma:%.2f|Epsilon:%.2f|EpIter:%.2f" % (mu_01, muIncr_01, beta_pos, sigma_pos, epsilon_pos, epsilonIterate_01))
    
    with HiddenPrints():
        f1=entropy_F1(beta=beta_pos,
                      epsilon=epsilon_pos,
                      epsilonIterate=epsilonIterate_01,
                      muIncr=muIncr_01,
                      sigma=sigma_pos,
                      mu=mu_01)
    
    return logit(f1)

In [16]:
entropy_F1_trx(beta_sp=inv_softplus(33),
               sigma_sp=inv_softplus(449),
               mu_logit=logit(0.5),
               muIncr_logit=logit(0.05),
               epsilon_sp=inv_softplus(31.8))

Mu:0.50|Mu+:0.05|Beta:33.00|Sigma:449.00|Epsilon:31.80|EpIter:0.00


1.1173424739923359

In [17]:
optimizer=None

In [18]:
pbounds = {'beta_sp': (inv_softplus(1), inv_softplus(700)), 
           'sigma_sp': (inv_softplus(1), inv_softplus(700)),
           'mu_logit': (logit(0.5), logit(0.99)),
           'muIncr_logit': (logit(0.01), logit(0.2)),
           'epsilon_sp': (inv_softplus(1), inv_softplus(700))}#,
           #'epsilonIterate_logit': (logit(0), logit(0.25))}

In [19]:
from bayes_opt import SequentialDomainReductionTransformer

In [20]:
bounds_transformer = SequentialDomainReductionTransformer()

In [21]:
optimizer = BayesianOptimization(
    f=entropy_F1_trx,
    pbounds=pbounds,
    random_state=1,
    bounds_transformer=bounds_transformer
)

In [21]:
# optimizer.probe(
#     params={"beta_sp": inv_softplus(14), 
#             "sigma_sp": inv_softplus(24),
#             "mu_logit":logit(0.5),
#             #'epsilonIterate_logit': logit(0),
#             "muIncr_logit":logit(0.1),
#             "epsilon_sp":inv_softplus(3.0)},
#     lazy=True,
# )

In [23]:
optimizer.maximize(
    init_points=10,
    n_iter=50,
    acq="ucb",
    kappa=5,
    alpha=1
)

|   iter    |  target   |  beta_sp  | epsilo... | muIncr... | mu_logit  | sigma_sp  |
-------------------------------------------------------------------------------------
Mu:0.80|Mu+:0.01|Beta:292.00|Sigma:103.00|Epsilon:504.38|EpIter:0.00
|  1        | -0.7735   |  292.2    |  504.4    | -4.595    |  1.389    |  103.2    |
Mu:0.86|Mu+:0.03|Beta:65.00|Sigma:377.00|Epsilon:130.82|EpIter:0.00
|  2        | -1.313    |  65.13    |  130.8    | -3.486    |  1.823    |  377.4    |
Mu:0.98|Mu+:0.02|Beta:293.00|Sigma:19.00|Epsilon:479.82|EpIter:0.00
|  3        | -3.469    |  293.8    |  479.8    | -3.939    |  4.035    |  19.7     |
Mu:0.66|Mu+:0.06|Beta:469.00|Sigma:139.00|Epsilon:292.43|EpIter:0.00
|  4        |  1.118    |  469.5    |  292.4    | -2.802    |  0.6451   |  139.1    |
Mu:0.96|Mu+:0.03|Beta:560.00|Sigma:613.00|Epsilon:677.80|EpIter:0.00
|  5        | -2.848    |  560.6    |  677.8    | -3.589    |  3.181    |  613.5    |
Mu:0.69|Mu+:0.01|Beta:626.00|Sigma:614.00|Epsilon:60.03

|  39       |  1.054    |  490.3    |  272.6    | -2.717    |  0.5083   |  118.3    |
Mu:0.64|Mu+:0.06|Beta:489.00|Sigma:118.00|Epsilon:273.04|EpIter:0.00
|  40       |  1.276    |  489.8    |  273.0    | -2.719    |  0.5689   |  118.8    |
Mu:0.63|Mu+:0.06|Beta:489.00|Sigma:127.00|Epsilon:273.47|EpIter:0.00
|  41       |  1.191    |  489.4    |  273.5    | -2.721    |  0.5144   |  127.1    |
Mu:0.64|Mu+:0.06|Beta:488.00|Sigma:119.00|Epsilon:280.75|EpIter:0.00
|  42       |  1.278    |  489.0    |  280.7    | -2.723    |  0.5635   |  119.6    |
Mu:0.63|Mu+:0.06|Beta:482.00|Sigma:119.00|Epsilon:274.19|EpIter:0.00
|  43       |  1.04     |  482.3    |  274.2    | -2.724    |  0.5193   |  120.0    |
Mu:0.64|Mu+:0.06|Beta:488.00|Sigma:126.00|Epsilon:274.50|EpIter:0.00
|  44       |  1.267    |  488.3    |  274.5    | -2.726    |  0.5591   |  126.0    |
Mu:0.64|Mu+:0.06|Beta:488.00|Sigma:120.00|Epsilon:279.81|EpIter:0.00
|  45       |  1.289    |  488.0    |  279.8    | -2.727    |  0.5572 

In [34]:
optimizer.max

{'target': 1.2963461271957242,
 'params': {'beta_sp': 485.4456526355273,
  'epsilon_sp': 277.297930153227,
  'muIncr_logit': -2.7372215941927043,
  'mu_logit': 0.5402285057903428,
  'sigma_sp': 123.15064699252713}}

In [35]:
inv_logit(optimizer.max["target"])

0.7852194000000001

In [36]:
params=optimizer.max["params"]

In [37]:
softplus(params["beta_sp"])

485.4456526355273

In [38]:
softplus(params["sigma_sp"])

123.15064699252713

In [39]:
inv_logit(params["mu_logit"])

0.6318655724055106

In [40]:
inv_logit(params["muIncr_logit"])

0.060812396374509706

In [41]:
softplus(params["epsilon_sp"])

277.297930153227

In [43]:
entropy_F1(mu=0.6319,
           muIncr=0.0608,
           beta=485,
           sigma=123,
           epsilon=277.30,
           epsilonIterate=0)


>>>>>>>>>>>>
Starting Iteration mu= 0.6319
Starting Iteration epsilon= 277.3
Running NewMatrix Jar!
 Single-Reference Blocks = 1

 Multi-Reference Blocks = 1802
NewMatrix Iterateblocks Total Time = 7.230738596059382

Precision = TP / (TP + FP) ......................... = 0.78230088
Recall = TP / (TP + FN) ............................ = 0.78815977
F-Measure .......................................... = 0.78521940


0.7852194